#### RAG (Retrieval-Augmented Generation)
- 모델의 학습 데이터에 포함되지 않은 데이터를 사용 (환각 방지)
- **외부 데이터**를 검색(retrieval)한 후, 생성(generation) 단계에서 LLM에 전달
- 모델은 주어진 컨텍스트나 질문에 더 적합하고 풍부한 정보를 반영한 답변을 생성
- 논문: https://arxiv.org/abs/2005.11401

## 0. 환경 구성

### 1) 라이브러리 설치

In [1]:
%pip install -q langchain langchain-openai langchain_community tiktoken chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
import langchain

langchain.__version__

'0.3.21'

### 2) OpenAI 인증키 설정
https://openai.com/

In [3]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

## 1. RAG 파이프라인 개요
- Load Data - Text Split - Indexing - Retrieval - Generation



##### Step 1: Load Data
###### 1. WebBaseLoader를 사용하여 웹 페이지 데이터 가져오기

In [ ]:
import os
from langchain_community.document_loaders import WebBaseLoader

# 웹 요청을 위한 USER_AGENT 환경 변수 설정 (필요한 경우)
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"

# 환경 변수 확인
print(f"현재 설정된 USER_AGENT: {os.environ.get('USER_AGENT')}")

# 웹페이지 URL 지정  https://ko.wikipedia.org/wiki/축구_경기_규칙
url = 'https://ko.wikipedia.org/wiki/%EC%B6%95%EA%B5%AC_%EA%B2%BD%EA%B8%B0_%EA%B7%9C%EC%B9%99'

# WebBaseLoader 초기화 및 데이터 로드
loader = WebBaseLoader(url)
docs = loader.load()

# 로드된 문서 확인
print(type(docs), len(docs))
print(docs)
print(type(docs[0]))  # <class 'langchain_core.documents.Document'>


In [ ]:
print(len(docs[0].page_content))
print(docs[0].page_content[5000:5500])
print(docs[0].metadata)

##### Step 2: 문서 분할(Splitting)

* WebBaseLoader를 사용하여 웹 페이지에서 가져온 데이터를 RAG 시스템에서 효율적으로 활용하기 위해 작은 청크(chunks)로 분할합니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)

# 문서 분할
splits = text_splitter.split_documents(docs)

# 분할된 문서 확인
print(type(splits), len(splits))  # 총 몇 개의 청크가 생성되었는지 확인
print(type(splits[0]))
print(splits[0].page_content[:20])  # 첫 번째 청크의 일부 출력

In [ ]:
# 열번째 청크의 내용 출력
print(splits[10].page_content[:20])
# 열번째 청크의 메타데이터 출력
print(splits[10].metadata)

##### Step 3: 벡터 DB에 저장 및 검색
* Indexing (Texts -> Embedding -> Store)

In [ ]:
from langchain_community.vectorstores import Chroma  # 벡터 저장소 라이브러리
from langchain_openai import OpenAIEmbeddings # OpenAI의 임베딩(Embedding) 모델

# 1. Chroma 벡터 저장소 생성
# - documents: 텍스트 데이터를 벡터화 하여 저장할 문서 리스트
# - embedding: 문서를 벡터로 변환하는 OpenAI Embeddings 모델 사용
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 2. 유사 문서 검색 (Similarity Search)
# - "경기장 표시에 대해서 설명해주세요."라는 쿼리에 대해,
# - Chroma 벡터 저장소에서 가장 유사한 문서를 검색함.
docs = vectorstore.similarity_search("경기장 표시에 대해서 설명해주세요.")

# 3. 검색된 문서의 타입과 개수 출력
print(type(docs), len(docs))
# 4. 검색된 첫 번째 문서 내용 출력
print(docs[0].page_content)

##### Step 4: RAG Pipeline을 이용한 질의응답 시스템 구축

In [ ]:
from langchain_openai import ChatOpenAI  # OpenAI LLM(대화형 언어 모델)
from langchain_core.prompts import ChatPromptTemplate  # 프롬프트 템플릿
from langchain_core.runnables import RunnablePassthrough  # 입력을 그대로 전달하는 유틸리티
from langchain_core.output_parsers import StrOutputParser  # LLM 응답을 문자열로 변환하는 파서

# 검색 개수 제한 설정
# - 벡터 저장소(vectorstore)에서 관련성이 높은 문서 최대 3개를 검색하도록 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  

# 문서 포맷 변환 함수 (요약 포함)
# def format_docs(docs):
#     summaries = [doc.page_content[:300] + "..." for doc in docs]  
#     # - 각 문서의 내용을 앞부분 300자까지만 가져와 요약 (가독성을 위해 "..." 추가)
#     return "\n\n".join(summaries)  
#     # - 검색된 여러 문서를 하나의 문자열로 합침 (문서 간 개행 추가)

def format_docs(docs):
    summaries = [f"출처: {doc.metadata.get('source', '알 수 없음')}\n" + doc.page_content[:300] + "..." for doc in docs]
    return "\n\n".join(summaries)    

# 프롬프트 템플릿 설정
# - 모델이 주어진 `context`(검색된 문서)만을 참고하여 질문에 답변하도록 유도하는 프롬프트 템플릿
# - {context}: 검색된 문서 요약이 삽입될 자리
# - {question}: 사용자의 질문이 삽입될 자리
template = '''당신은 제공된 컨텍스트를 기반으로 질문에 답하는 AI 어시스턴트입니다. 
반드시 컨텍스트 내 정보를 활용하여 정확하고 신뢰할 수 있는 답변을 제공하세요.

[컨텍스트]
{context}

[질문]
{question}

[답변]
'''

# - 위에서 정의한 템플릿을 사용하여 LangChain의 프롬프트 객체 생성
prompt = ChatPromptTemplate.from_template(template)  

# LLM 모델 설정
# - OpenAI의 `gpt-3.5-turbo-0125` 모델 사용 (정확도를 높이기 위해 temperature=0 설정)
model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

# RAG 체인 설정
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  
    # - `retriever`를 통해 검색된 문서를 `format_docs()` 함수로 가공하여 `context`로 전달
    # - `question`은 변형 없이 그대로 전달
    | prompt  # - 위에서 정의한 `ChatPromptTemplate`을 적용
    | model   # - OpenAI GPT-3.5 모델을 사용해 응답 생성
    | StrOutputParser()  # - 모델의 응답을 문자열로 변환
)

# 실행 (사용자 질문을 입력으로 받아 RAG 체인 실행)
# - "경기장 표시에 대해서 설명해주세요."라는 질문을 LLM에 전달하여 답변을 생성
response = rag_chain.invoke("경기장 표시에 대해서 설명해주세요.")  

# 최종 응답 출력
print(f" 모델 응답:\n{response}")